In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import csv
import numpy as np
import sklearn
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##considering make another function

In [ ]:
years = [year for year in range(2021, 2009, -1)]
years

[2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]

In [ ]:
from re import I
def process_excel_file(year, excel_path, list_index):
    df = pd.read_excel(excel_path, sheet_name=1, header=None)

    header = df.iloc[0:3].apply(lambda x: '_'.join(x.astype(str)), axis=0)
    df.columns = header
    df = df[3:]

    df = df.set_index(df.columns[0])
    df = df.transpose()

    df = df[df.reset_index().index % 2 == 0]

    # Calculate the starting and ending row indices based on the list_index
    start_row = list_index * 4
    end_row = (list_index + 1) * 4

    new_df = df.iloc[start_row:end_row].copy()
    new_df.insert(0, 'year', year)
    new_df = new_df.reset_index()

    if isinstance(new_df.iat[0, 0], str) and len(new_df.iat[0, 0]) > 23:
        new_df.iat[0, 0] = new_df.iat[0, 0][-23:]

    new_df = new_df.rename(columns={'index': 'estimate_type'})
    new_df['estimate_type'] = new_df['estimate_type'].str[4:]
    new_df['Total'] = pd.to_numeric(new_df['Total'].str.replace(',', ''), errors='coerce').astype(float)

    return new_df


In [ ]:
list_of_indices = ["901", "902", "903", "907",
                   "90901", "91001",
                   "911", "912", "913", "914", "915", "916", "917", "918", "919", "920",
                   "92101", "922", "923", "924",
                   "1001", "1002", "1003", "1004", "1005", "100601", "100603", "1007", "1008"]



  #['70801', '70901', '801', '803', '80401', '805', '80601',
                   #'81302', '814', '815', '817', '818', '819', '820', '821', '904','906', '9803']


In [ ]:
len(list_of_indices)

29

In [ ]:
#this loop makes use of the function process_excel_file() to make different years of dataframe for each census tract
for index in list_of_indices:
    for year in years:
        excel_file_path = f'/content/drive/MyDrive/DS project/Dorchester_income/ACSST5Y{year}.xlsx'
        variable_name = f'ct{index}_{year}'

        # Determine the list_index based on the current index
        list_index = list_of_indices.index(index)

        # Call the function and assign the result to the variable
        globals()[variable_name] = process_excel_file(year, excel_file_path, list_index)


In [ ]:
#this for loop generates the concated dataframe

for index in list_of_indices:
    concatenated_dfs = [globals()[f'ct{index}_2021']]  # Start with the 2021 DataFrame

    for year in years[1:]:
        variable_name = f'ct{index}_{year}'
        if not globals()[variable_name].empty:
            concatenated_dfs.append(globals()[variable_name])

    # Concatenate all DataFrames for the current index
    globals()[f'ct{index}'] = pd.concat(concatenated_dfs, ignore_index=True)

In [ ]:
ct1001

nan_nan_Label,estimate_type,year,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars),PERCENT ALLOCATED,Household income in the past 12 months,Family income in the past 12 months,Nonfamily income in the past 12 months,PERCENT IMPUTED
0,Households_Estimate,2021,2124.0,12.9%,10.5%,5.7%,18.5%,10.9%,16.4%,4.2%,7.5%,7.9%,5.4%,"36,471","62,289",NaN,43.6%,(X),(X),NaN
1,Families_Estimate,2021,1639.0,5.7%,5.7%,3.2%,24.8%,14.2%,16.4%,5.5%,11.6%,7.1%,5.9%,"41,414","69,180",NaN,(X),43.7%,(X),NaN
2,Married-couple families_Estimate,2021,581.0,0.0%,0.0%,2.1%,7.4%,19.8%,18.2%,6.5%,24.8%,12.7%,8.4%,"84,609",N,NaN,(X),(X),(X),NaN
3,Nonfamily households_Estimate,2021,485.0,36.9%,26.8%,14.4%,4.9%,0.0%,8.9%,0.0%,4.3%,0.0%,3.7%,"13,401","31,374",NaN,(X),(X),40.4%,NaN
4,Households_Estimate,2020,2100.0,15.3%,9.3%,13.0%,15.1%,11.1%,10.0%,7.5%,12.0%,2.8%,3.8%,"31,071","53,702",NaN,39.7%,(X),(X),NaN
5,Families_Estimate,2020,1644.0,8.6%,4.7%,14.0%,20.4%,12.2%,10.5%,8.2%,14.2%,2.4%,4.8%,"35,969","60,033",NaN,(X),41.6%,(X),NaN
6,Married-couple families_Estimate,2020,507.0,0.0%,1.8%,2.0%,11.6%,18.7%,6.7%,11.0%,29.0%,7.9%,11.2%,"96,313",N,NaN,(X),(X),(X),NaN
7,Nonfamily households_Estimate,2020,456.0,39.7%,25.9%,9.4%,3.9%,7.5%,0.2%,9.4%,3.9%,0.0%,0.0%,"11,546","24,584",NaN,(X),(X),29.2%,NaN
8,Households_Estimate,2019,2203.0,19.9%,11.7%,12.6%,11.9%,12.5%,9.8%,6.5%,10.3%,2.4%,2.4%,"28,659","48,088",NaN,37.7%,(X),(X),NaN
9,Families_Estimate,2019,1578.0,10.8%,8.2%,12.8%,17.6%,13.6%,9.6%,8.1%,13.8%,2.2%,3.4%,"35,272","56,651",NaN,(X),42.9%,(X),NaN


In [ ]:
def convert_percentage_to_float_with_symbol(percentage_str):
    if isinstance(percentage_str, str):
        cleaned_str = percentage_str.replace('%', '').replace('-', '')
        try:
            return float(cleaned_str) / 100.0
        except ValueError:
            # Handle cases where the cleaned string couldn't be converted to float
            return None
    else:
        return percentage_str


def convert_column_to_float(df):
  columns_to_convert = df.columns[3:13]

  for col in columns_to_convert:
    df[col] = df[col].apply(convert_percentage_to_float_with_symbol)
    df[col] = df[col] * df['Total']
  # Convert the selected columns to float64
  #df[columns_to_convert] = df[columns_to_convert].astype(float)
  return df

In [ ]:
#processing the concated dataframe, (strip%and, mutliplied by total)
for index in list_of_indices:
  dataframe_name = f'ct{index}'
  globals()[dataframe_name]  = convert_column_to_float(globals()[dataframe_name])


In [ ]:
for index in list_of_indices:
  dataframe_name = f'ct{index}'
  globals()[dataframe_name]  = globals()[dataframe_name][globals()[dataframe_name]['estimate_type']=='Households_Estimate']

In [ ]:
#calculating income difference
for index in list_of_indices:
    df = f'ct{index}'

    # Clean the 'Median income (dollars)' column by removing commas and dashes
    globals()[df]['Median income (dollars)'] = globals()[df]['Median income (dollars)'].str.replace(',', '').str.replace('-', '')
    # Clean the 'Mean income..
    globals()[df]['Mean income (dollars)'] = globals()[df]['Mean income (dollars)'].str.replace(',', '').str.replace('-', '')

    # Convert the cleaned column to float, handling non-numeric values
    def convert_to_float(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return None

    globals()[df]['Median income (dollars)'] = globals()[df]['Median income (dollars)'].apply(convert_to_float)
    globals()[df]['Mean income (dollars)'] = globals()[df]['Mean income (dollars)'].apply(convert_to_float)

    # Calculate the 'median_income_difference' column
    globals()[df]['median_income_difference'] = ((globals()[df]['Median income (dollars)'] - globals()[df]['Median income (dollars)'].shift(-1)) / globals()[df]['Median income (dollars)'].shift(-1))
    globals()[df]['mean_income_difference'] = ((globals()[df]['Mean income (dollars)'] - globals()[df]['Mean income (dollars)'].shift(-1)) / globals()[df]['Mean income (dollars)'].shift(-1))



<ipython-input-48-1f052636bfbd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[df]['Median income (dollars)'] = globals()[df]['Median income (dollars)'].str.replace(',', '').str.replace('-', '')
<ipython-input-48-1f052636bfbd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[df]['Mean income (dollars)'] = globals()[df]['Mean income (dollars)'].str.replace(',', '').str.replace('-', '')
<ipython-input-48-1f052636bfbd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [ ]:
# adding census tract identifier column
for index in list_of_indices:
  dataframe_name = f'ct{index}'
  globals()[dataframe_name]['census_tract']  = index

<ipython-input-49-e561fdeb9973>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[dataframe_name]['census_tract']  = index


In [ ]:

list_of_indices
# Create a list of DataFrames
concatenated_df = [globals()[f'ct{index}'] for index in list_of_indices]

# Initialize an empty DataFrame to store the concatenated result
dorchester_income = pd.DataFrame()

# Concatenate the DataFrames in the list
for df in concatenated_df:
    dorchester_income = pd.concat([dorchester_income, df], ignore_index=True)

In [ ]:
dorchester_income.columns

Index(['estimate_type', 'year', 'Total', 'Less than $10,000',
       '$10,000 to $14,999', '$15,000 to $24,999', '$25,000 to $34,999',
       '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
       'Median income (dollars)', 'Mean income (dollars)', 'PERCENT ALLOCATED',
       'Household income in the past 12 months',
       'Family income in the past 12 months',
       'Nonfamily income in the past 12 months', 'PERCENT IMPUTED',
       'median_income_difference', 'mean_income_difference', 'census_tract'],
      dtype='object', name='nan_nan_Label')

In [ ]:
income_dorchester_result_df = dorchester_income[['year', 'Median income (dollars)', 'Mean income (dollars)','median_income_difference',
                                                 'mean_income_difference', 'census_tract']]

#income_dorchester_result_df = income_dorchester_result_df[income_dorchester_result_df['census_tract'] != 9803]
income_dorchester_result_df = income_dorchester_result_df.reset_index(drop=True)


In [ ]:
income_dorchester_result_df['census_tract'] = income_dorchester_result_df['census_tract'].astype('float64')

In [ ]:
# Adding back the decimal to census tract..
def custom_replace(value):
    if value > 10000:
        return value/100
   # else: return int(value)

    return value


income_dorchester_result_df['census_tract'] = income_dorchester_result_df['census_tract'].apply(custom_replace)

In [ ]:
income_dorchester_result_df

nan_nan_Label,year,Median income (dollars),Mean income (dollars),median_income_difference,mean_income_difference,census_tract
0,2021,33652.0,55348.0,0.017630,0.106739,901.0
1,2020,33069.0,50010.0,-0.032929,-0.105287,901.0
2,2019,34195.0,55895.0,0.045655,0.185598,901.0
3,2018,32702.0,47145.0,0.079880,0.076542,901.0
4,2017,30283.0,43793.0,0.067731,0.064384,901.0
...,...,...,...,...,...,...
343,2014,65762.0,73137.0,-0.043183,-0.014764,1008.0
344,2013,68730.0,74233.0,0.022905,-0.014680,1008.0
345,2012,67191.0,75339.0,0.104879,0.054297,1008.0
346,2011,60813.0,71459.0,0.070651,0.043990,1008.0


In [ ]:
income_dorchester_result_df.to_csv('/content/drive/MyDrive/DS project/Dorchester_income/Dorchester_income.csv')